In [2]:
import requests
import base64 
import json

In [ ]:
create_tracking={ #####--------------- Ini adalah payload yang direquest oleh Pak Asep -------------------########
  "plat_nomor": "B1083UFJ",
  "load_id": "67898787",
  "status ": "OPEN",
  "timestamp ": "2025-02-12 12:00:00",
  "list_order": [
    {
      "order_no": "TST20193012",
      "pick": {
        "seq": "1",
        "geofence_name": "CIKARANG BARAT"
      },
      "drop": {
        "seq": "1",
        "geofence_name": "BENDA"
      }
    },
    {
      "order_no": "TST20193013",
      "pick": {
        "seq": "2",
        "geofence_name": "CIKARANG UTARA"
      },
      "drop": {
        "seq": "2",
        "geofence_name": "BENDA"
      }
    }
  ]
}

create_tracking_json=json.dumps(create_tracking)
print(create_tracking)

{'plat_nomor': 'B1083UFJ', 'load_id': '67898787', 'status ': 'OPEN', 'timestamp ': '2025-02-12 12:00:00', 'list_order': [{'order_no': 'TST20193012', 'pick': {'seq': '1', 'geofence_name': 'CIKARANG BARAT'}, 'drop': {'seq': '1', 'geofence_name': 'BENDA'}}, {'order_no': 'TST20193013', 'pick': {'seq': '2', 'geofence_name': 'CIKARANG UTARA'}, 'drop': {'seq': '2', 'geofence_name': 'BENDA'}}]}


In [ ]:
def create_alert_name(load_id, order_no, action_type, seq): #### Fungsi ini membuat payload diatas menjadi alert name persatuan geofence ###
    return f"{load_id}-{order_no}-{action_type}-{seq}"

In [5]:
def get_auth_header(username, token):
    """Generate the authorization header from credentials"""
    auth = f"{username}:{token}"
    auth_bytes = auth.encode('ascii')
    base64_bytes = base64.b64encode(auth_bytes)
    base64_auth = base64_bytes.decode('ascii')
    
    return {
        'Authorization': f'Basic {base64_auth}',
        'Content-Type': 'application/json'
    }

In [ ]:
def get_multiple_geofence_ids(username, token, geofence_names):
    """Retrieve geofence IDs for a list of geofence names"""
    geofence_details = {}
    
    for geofence_name in geofence_names:
        # Skip if we've already fetched this geofence
        if geofence_name in geofence_details:
            continue
            
        url = f"https://fleetapi-id.cartrack.com/rest/geofences?limit=200&filter[name]={geofence_name}"
        headers = get_auth_header(username, token)
        
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching geofence '{geofence_name}': {response.status_code}")
            print(response.text)
            continue
        
        # Parse the response
        geofence_data = response.json().get('data', [])
        
        if not geofence_data:
            print(f"No geofence found with name '{geofence_name}'")
            continue
        
        # Find the exact match geofence name if possible -->> Checking the database whether it's geofence ID with the same geofence name
        exact_match = None
        for geofence in geofence_data:
            if geofence["name"].lower() == geofence_name.lower():
                exact_match = geofence
                break
        
        # Use exact match if found, otherwise use the first result
        selected_geofence = exact_match if exact_match else geofence_data[0]
        
        # Store the geofence details
        geofence_info = {
            "geofence_id": selected_geofence["geofence_id"],
            "name": selected_geofence["name"],
            "description": selected_geofence.get("description", "")
        }
        
        geofence_details[geofence_name] = geofence_info
    
    return geofence_details

In [ ]:
alert_mappings=[] #####------- this function categorize the alert name with specific payload requested -------------###########
load_id=create_tracking.get('load_id','')
registration=create_tracking.get('plat_nomor','')
geofence_names=set()
for order in create_tracking.get('list_order', []):
        order_no=order.get('order_no','')
        pick_info=order.get('pick',{})
        pick_geofence = pick_info.get('geofence_name')
        pick_seq=pick_info.get('seq','')

        if pick_geofence:
            geofence_names.add(pick_geofence)
            alert_name = create_alert_name(load_id, order_no, "PICK", pick_seq)
            alert_mappings.append({
                "original_geofence_name": pick_geofence,
                "alert_name": alert_name,
                "order_no": order_no,
                "action_type": "PICK",
                "seq": pick_seq,
                "geofence_id": None,
                "registration":registration
            })

        drop_info = order.get('drop', {})
        drop_geofence = drop_info.get('geofence_name')
        drop_seq = drop_info.get('seq', '')

        if drop_geofence:           
            # Create alert mapping for drop
            geofence_names.add(drop_geofence)
            alert_name = create_alert_name(load_id, order_no, "DROP", drop_seq)
            alert_mappings.append({
                "original_geofence_name": drop_geofence,
                "alert_name": alert_name,
                "order_no": order_no,
                "action_type": "DROP",
                "seq": drop_seq,
                "geofence_id": None,
                "registration":registration

            })
        username = "BAHA00004"
        token = "9328ccf602f78bfdabef61a6f0d748f35559346bac09f49f94c41284240f8e4a"
        geofence_details=get_multiple_geofence_ids(username,token,geofence_names)
        for mapping in alert_mappings:
            geofence_name = mapping["original_geofence_name"]
            if geofence_name in geofence_details:
                mapping["geofence_id"] = geofence_details[geofence_name]["geofence_id"]


    

In [ ]:
from pprint import pprint

pprint(alert_mappings) ###DEBUGGING###

[{'action_type': 'PICK',
  'alert_name': '67898787-TST20193012-PICK-1',
  'geofence_id': '08b003be-ef77-11ef-830a-506b8dfdcc6f',
  'order_no': 'TST20193012',
  'original_geofence_name': 'CIKARANG BARAT',
  'registration': 'B1083UFJ',
  'seq': '1'},
 {'action_type': 'DROP',
  'alert_name': '67898787-TST20193012-DROP-1',
  'geofence_id': '368a6076-f4dc-11ef-94e2-506b8dfdcc6f',
  'order_no': 'TST20193012',
  'original_geofence_name': 'BENDA',
  'registration': 'B1083UFJ',
  'seq': '1'},
 {'action_type': 'PICK',
  'alert_name': '67898787-TST20193013-PICK-2',
  'geofence_id': '0c132d2e-ef77-11ef-ad01-506b8dfdcc6f',
  'order_no': 'TST20193013',
  'original_geofence_name': 'CIKARANG UTARA',
  'registration': 'B1083UFJ',
  'seq': '2'},
 {'action_type': 'DROP',
  'alert_name': '67898787-TST20193013-DROP-2',
  'geofence_id': '368a6076-f4dc-11ef-94e2-506b8dfdcc6f',
  'order_no': 'TST20193013',
  'original_geofence_name': 'BENDA',
  'registration': 'B1083UFJ',
  'seq': '2'}]


In [ ]:
## This function is to create payload to be geofence alert ##

create_alert_url= "https://fleetapi-id.cartrack.com/rest/alerts/geofences"
results=[]
for index, alert in enumerate(alert_mappings):
    # Create a descriptive name combining action type, order number and sequence
    name = alert['alert_name']
    
    # Get the registration and convert it to a list (API expects a list)
    registrations = alert['registration']
    
    # Get the geofence ID
    geofence_id = alert['geofence_id']
    
    # Create the payload
    alert_payload = {
        "name": name,
        "registrations": [registrations],  # API expects a list of registrations
        "geofence_trigger_id": 3,  # 3 means enters and leaves the geofences
        "geofence_ids": [geofence_id],  # API expects a list of geofence IDs
        "geofence_group_ids": [],
        "contact_type": {
            "contact_type_id": 4,  # 4 means send notification to alert center
            "values":[],
            "priority_id": 1  # 1 means high priority
        }
    }
    
    # Convert the payload to JSON
    #json_payload = json.dumps(alert_payload)

    print(json.dumps(alert_payload, indent=2))
    response = requests.request("POST", create_alert_url, headers=get_auth_header(username,token), data=json.dumps(alert_payload))

    results.append({
        "index": index + 1,
        "alert_name": name,
        "geofence_id": geofence_id,
        "geofence_name": alert['original_geofence_name'],
        "status_code": response.status_code,
        "response":response.text
    })
    print(f"RESPONSE: {response.text}")

for result in results:
    print(f"{result['index']}. {result['alert_name']} - Geofence: {result['geofence_name']} - Status: {result['status_code']}")

{
  "name": "67898787-TST20193012-PICK-1",
  "registrations": [
    "B1083UFJ"
  ],
  "geofence_trigger_id": 3,
  "geofence_ids": [
    "08b003be-ef77-11ef-830a-506b8dfdcc6f"
  ],
  "geofence_group_ids": [],
  "contact_type": {
    "contact_type_id": 4,
    "values": [],
    "priority_id": 1
  }
}
RESPONSE: {"data":{"alert_id":"232e724c-fd51-11ef-b2d6-506b8dfdcc6f"},"meta":{"message":"Alert created successfully"}}
{
  "name": "67898787-TST20193012-DROP-1",
  "registrations": [
    "B1083UFJ"
  ],
  "geofence_trigger_id": 3,
  "geofence_ids": [
    "368a6076-f4dc-11ef-94e2-506b8dfdcc6f"
  ],
  "geofence_group_ids": [],
  "contact_type": {
    "contact_type_id": 4,
    "values": [],
    "priority_id": 1
  }
}
RESPONSE: {"data":{"alert_id":"2355ac22-fd51-11ef-be0b-506b8dfdcc6f"},"meta":{"message":"Alert created successfully"}}
{
  "name": "67898787-TST20193013-PICK-2",
  "registrations": [
    "B1083UFJ"
  ],
  "geofence_trigger_id": 3,
  "geofence_ids": [
    "0c132d2e-ef77-11ef-ad01-506